
# 3.4 AGB modeling and mapping at the tile scale


In [17]:
#----------------------------------------------#
# Installing packages
#----------------------------------------------#
install.packages("randomForest")
install.packages("raster")
install.packages("data.table")

#----------------------------------------------#
# loading packages and functions
#----------------------------------------------#
library(randomForest)
library(raster)
library(rgdal)
library(data.table)
source("~/above/biomass_models/boreal_functions.R")


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [18]:
#----------------------------------------------#
# AGB modeling and mapping
#----------------------------------------------#
maps<-mapBoreal(rds_models=rds_models,
                models_id=models_id,
                ice2_30_atl08_path=ice2_30_atl08_path, 
                offset=0,
                s_train=70, 
                rep=10,
                stack=stack,
                strat_random=TRUE,
                output=output)

plot(maps)

  |                                                                      |   0%

ERROR: Error in as.data.frame.default(stack, xy = TRUE): cannot coerce class ‘structure("standardGeneric", package = "methods")’ to a data.frame


In [22]:
#mapBoreal<-function(rds_models,
#                    models_id,
#                    ice2_30_atl08_path, 
                    offset=0
                    s_train=70 
                    rep=10
#                    stack,
#                    strat_random=TRUE,
#                    output){
    
  
  # apply GEDI models  
  xtable<-GEDI2AT08AGB(rds_models=rds_models,
                       models_id=models_id,
                       ice2_30_atl08_path=ice2_30_atl08_path, 
                       offset=offset)
  
  print(str(xtable))
  # run 
  maps<-agbModelingMapping(x=xtable[,c(15:18,20:34,36:37)],
                           y=xtable$AGB,
                           se=xtable$SE,
                           s_train=s_train,
                           rep=rep,
                           stack=stack,
                           strat_random=strat_random)

  writeRaster(maps,output,overwrite=T)
#  return(maps)
#}

'data.frame':	270 obs. of  40 variables:
 $ lon      : num  -117 -117 -117 -117 -117 ...
 $ lat      : num  51.1 51.1 51.1 51 51 ...
 $ rh25     : num  4.954 11.875 3.24 0.857 1.234 ...
 $ rh50     : num  17.62 17.97 7.39 1.63 2.34 ...
 $ rh60     : num  19 19.32 9.18 2.07 2.88 ...
 $ rh70     : num  22.36 21.32 11.44 2.94 4.45 ...
 $ rh75     : num  23.11 22.36 14.28 3.1 5.36 ...
 $ rh80     : num  24.37 22.87 16.07 3.48 6.16 ...
 $ rh85     : num  26.9 24.52 18.09 4.06 7.81 ...
 $ rh90     : num  28.88 26.04 20.32 5.62 8.69 ...
 $ rh95     : num  31.2 27.7 22.1 10.4 11 ...
 $ h_can    : num  32.2 28.6 22.9 12 11.6 ...
 $ h_max_can: num  33.1 30.4 23.5 15.3 14.8 ...
 $ geometry : Factor w/ 54 levels "POINT (3786849.033514066 3144476.935574841)",..: 16 15 39 42 43 44 45 46 47 48 ...
 $ elevation: num  852 873 0 1213 1216 ...
 $ slope    : num  16.35 17.59 0 3.21 2.15 ...
 $ tsri     : num  0.92001 0.86621 0 0.09859 0.00759 ...
 $ tpi      : num  0.74341 0.00673 0 -0.14805 0.00359 ...
 

ERROR: Error in as.data.frame.default(stack, xy = TRUE): cannot coerce class ‘structure("standardGeneric", package = "methods")’ to a data.frame


In [30]:
#agbModelingMapping<-function(x=x,y=y,se=NULL,s_train=70, rep=10,stack=stack,strat_random=TRUE,output){
 x=xtable[,c(15:18,20:34,36:37)]
y=xtable$AGB
se=xtable$SE
s_train=s_train
rep=rep
stack=stack
strat_random=TRUE

  pb <- txtProgressBar(min = 0, max = rep, style = 3)
  str(pb)
  stats_df<-NULL
  n<-length(x)
  ids<-1:n
  map_pred<-NULL
  
  #p_load("VSURF")
  #varsel <- VSURF(y=y, x=x, ntree=1000)
  #varnames<-colnames(x)[varsel$varselect.interp]
  stack_df <- na.omit(as.data.frame(stack, row.names=FALSE, xy=TRUE))
  stack_df$grid_id<-1:nrow(stack_df)
  #i=10
  i.s=0
  #j=1
  
  for ( j in 1:rep){
    i.s<-i.s+1
    setTxtProgressBar(pb, i.s)
    
    set.seed(j)
    if ( strat_random==TRUE){
      trainRowNumbers<-stratRandomSample(agb=y,breaks=quantile(y, na.rm=T), p=s_train/100)
    } else {
      trainRowNumbers<-sort(sample(ids,round(n*s_train/100), T))
    }
    
    # Step 2: Create the training  dataset
    # select % of data to training and testing the models
    trainData.x <- x[trainRowNumbers,]
    trainData.y <- y[trainRowNumbers]
    
    
    # Step 3: Create the test dataset
    # select % of the data for validation
    testData.x <- x[!row.names(x) %in% trainRowNumbers,]
    testData.y <- y[!row.names(x) %in% trainRowNumbers]
    
    # rf modeling
    set.seed(j)
    fit.rf <- randomForest(y=trainData.y, x=trainData.x, ntree=1000)
    pred.rf<-predict(fit.rf, newdata=testData.x) # 
    #fit.rf <- randomForest(y=trainData.y, x=trainData.x[,varnames], ntree=1000)
    #pred.rf<-predict(fit.rf, newdata=testData.x[,varnames]) # 
    stats.rf<-cbind(method=rep("RF",6), rep=rep(j,6), StatModel(testData.y,pred.rf))
    
    # model validation stats
    stats_df<-rbind(stats_df,
                    stats.rf)
    row.names(stats_df)<-1:nrow(stats_df)
    
    # mapping
    #map_i<-cbind(stack_df[,1:2],agb=predict(fit.rf, newdata=stack_df[,varnames]), rep=i)
    map_i<-cbind(stack_df[,1:2],agb=predict(fit.rf, newdata=stack_df), rep=j, grid_id=stack_df$grid_id)
    map_pred<-rbind(map_pred,map_i)
    
  }
  
  
  mean_map<-tapply(map_pred$agb,map_pred$grid_id,mean)
  sd_map<-tapply(map_pred$agb,map_pred$grid_id,sd)
  
  
  agb_maps <- rasterFromXYZ(cbind(stack_df[,1:2],
                                  agb_mean=mean_map,
                                  agb_sd=sd_map,
                                  armse_map_mean=mean(stats_df[stats_df[,3]=="rmse",4]),
                                  armse_map_sd=sd(stats_df[stats_df[,3]=="rmse",4]),
                                  
                                  rrmse_map_mean=mean(stats_df[stats_df[,3]=="rmseR",4]),
                                  rrmse_map_sd=sd(stats_df[stats_df[,3]=="rmseR",4]),
                                  
                                  abias_map_mean=mean(stats_df[stats_df[,3]=="bias",4]),
                                  abias_map_sd=sd(stats_df[stats_df[,3]=="bias",4]),
                                  
                                  rbias_map_mean=mean(stats_df[stats_df[,3]=="bias",4]),
                                  rbias_map_sd=sd(stats_df[stats_df[,3]=="bias",4]),
                                  
                                  r2_map_mean=mean(stats_df[stats_df[,3]=="r2",4]),
                                  r2_map_sd=sd(stats_df[stats_df[,3]=="r2",4])))
  
  close(pb)
#  return(agb_maps)
#}

  |                                                                      |   0%List of 3
 $ getVal:function ()  
 $ up    :function (value)  
 $ kill  :function ()  
 - attr(*, "class")= chr "txtProgressBar"


ERROR: Error in as.data.frame.default(stack, row.names = FALSE, xy = TRUE): cannot coerce class ‘structure("standardGeneric", package = "methods")’ to a data.frame


In [38]:
str(stack)
#stack_df <- data.frame(stack)


Formal class 'standardGeneric' [package "methods"] with 8 slots
  ..@ .Data     :function (x, ...)  
  ..@ generic   : chr "stack"
  .. ..- attr(*, "package")= chr "utils"
  ..@ package   : chr "utils"
  ..@ group     : list()
  ..@ valueClass: chr(0) 
  ..@ signature : chr "x"
  ..@ default   :Formal class 'derivedDefaultMethod' [package "methods"] with 4 slots
  .. .. ..@ .Data  :function (x, ...)  
  .. .. ..@ target :Formal class 'signature' [package "methods"] with 3 slots
  .. .. .. .. ..@ .Data  : chr "ANY"
  .. .. .. .. ..@ names  : chr "x"
  .. .. .. .. ..@ package: chr "methods"
  .. .. ..@ defined:Formal class 'signature' [package "methods"] with 3 slots
  .. .. .. .. ..@ .Data  : chr "ANY"
  .. .. .. .. ..@ names  : chr "x"
  .. .. .. .. ..@ package: chr "methods"
  .. .. ..@ generic: chr "stack"
  .. .. .. ..- attr(*, "package")= chr "utils"
  ..@ skeleton  : language (new("derivedDefaultMethod", .Data = function (x, ...)  UseMethod("stack"), target = new("signature", .Dat

In [ ]:
GEDI2AT08AGB<-function(rds_models,models_id,ice2_30_atl08_path, offset=0){
  
  # rds_models
  
  names(rds_models)<-models_id
  # read table
  xtable<-read.table(ice2_30_atl08_path, sep=",", head=T)
  xtable_i<-na.omit(as.data.frame(xtable))
  names(xtable_i)[1:12]<-c("lon","lat","RH_25","RH_50","RH_60","RH_70","RH_75","RH_80","RH_85","RH_90","RH_95","RH_98")
  
  xtable_sqrt<-sqrt(xtable_i[3:12])
  names(xtable_sqrt)<-paste0("sqrt(",names(xtable_sqrt),")")
  xtable_sqrt<-cbind(xtable_i,xtable_sqrt)
  
  # get unique ids
  ids<-unique(xtable_sqrt$model_id)
  
  
  # apply models by id
  xtable_sqrt$AGB<-NA
  xtable_sqrt$SE<-NA
  
  for ( i in ids){
    
    # subset data for model id
    model_i<-readRDS(rds_models[names(rds_models)==i])
    
    # SE
    xtable_sqrt$SE[xtable_sqrt$model_id==i] <- summary(model_i)$sigma
    
    # AGB prediction
    xtable_sqrt$AGB[xtable_sqrt$model_id==i]<-predict(model_i, newdata=xtable_sqrt[xtable_sqrt$model_id==i,])
    
    #adjust for offset in model fits (note, this was 20 for ages, and now it's 100; essentially we added to all the height metrics so they would never be negative)
    xtable_sqrt$AGB[xtable_sqrt$model_id==i] <- xtable_sqrt$AGB[xtable_sqrt$model_id==i]+offset
    
    #define C
    C <- mean(model_i$fitted.values)/mean(model_i$model$`sqrt(AGBD)`)
    
    #we multiply by C in case there is a systematic over or under estimation in the model (bias correction)
    xtable_sqrt$AGB[xtable_sqrt$model_id==i]<-C*xtable_sqrt$AGB[xtable_sqrt$model_id==i]^2
    #print(head(xtable_sqrt$AGB[xtable_sqrt$model_id==i]))
  }
  xtable2<-xtable_sqrt[,c(names(xtable_i),"AGB","SE")]
  colnames(xtable2)[3:12]<-colnames(xtable)[3:12]
  return(xtable2)
}